In [12]:
!pip install roboflow

In [15]:
# Importing needed libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.amp import GradScaler, autocast
from sklearn.metrics import precision_score, recall_score, f1_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import numpy as np

# Setting device (Used A100 GPU for training)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:

# Using a dataset version from roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="zhU6ekpmZzulN3PMtVwd")
project = rf.workspace("debristech").project("campus-vision")
version = project.version(1)
dataset = version.download("folder")

!ls campus-vision-1


loading Roboflow workspace...
loading Roboflow project...
README.roboflow.txt  test  train  valid


In [17]:
# Set the directories for training and validation datasets
train_dir = "campus-vision-1/train"  # Path to training images
val_dir = "campus-vision-1/valid"    # Path to validation images

# Define the batch size for loading images
batch_size = 32  # Number of images processed in each batch

# Define the target image size for resizing
image_size = 512  # Resize images to 512x512 pixels

# Define transformations for the training set to augment and preprocess images
train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),         # Resize images to the target size
    transforms.RandomHorizontalFlip(),                   # Randomly flip images horizontally
    transforms.RandomRotation(10),                       # Randomly rotate images up to 10 degrees
    transforms.ColorJitter(                              # Randomly adjust brightness, contrast, saturation, and hue
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.1
    ),
    transforms.ToTensor(),                               # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406],          # Normalize using ImageNet's mean and std
                         [0.229, 0.224, 0.225])
])

# Define transformations for the validation set to preprocess images
val_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),         # Resize images to the target size
    transforms.ToTensor(),                               # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406],          # Normalize using ImageNet's mean and std
                         [0.229, 0.224, 0.225])
])



In [18]:
# Create a dataset for the training images, applying the training transformations
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)

# Create a dataset for the validation images, applying the validation transformations
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

# Create a DataLoader for the training set
train_loader = DataLoader(
    train_dataset,         # Dataset object for training
    batch_size=batch_size, # Number of images per batch
    shuffle=True           # Shuffle images to randomize the order in each epoch
)

# Create a DataLoader for the validation set
val_loader = DataLoader(
    val_dataset,           # Dataset object for validation
    batch_size=batch_size, # Number of images per batch
    shuffle=False          # No shuffling needed for validation to maintain order
)

In [19]:

# Initialize ResNet50 and move it to device
num_classes = len(train_dataset.classes)
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


In [20]:
# Define loss function, optimizer, and GradScaler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scaler = torch.amp.GradScaler(device='cuda')


In [21]:
# Create a dataset for the training images, applying the training transformations
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)

# Create a dataset for the validation images, applying the validation transformations
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

# Create a DataLoader for the training set
train_loader = DataLoader(
    train_dataset,         # Dataset object for training
    batch_size=batch_size, # Number of images per batch
    shuffle=True           # Shuffle images to randomize the order in each epoch
)

# Create a DataLoader for the validation set
val_loader = DataLoader(
    val_dataset,           # Dataset object for validation
    batch_size=batch_size, # Number of images per batch
    shuffle=False          # No shuffling needed for validation to maintain order
)




In [22]:
# Training function with mixed precision
def train_epoch(model, train_loader):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Use autocast for mixed precision
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Scale loss and update model parameters
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = correct / total
    return train_loss, train_acc

# Validation function with metrics and log_loss fix
def validate_epoch(model, val_loader):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(torch.softmax(outputs, dim=1).cpu().numpy())

    val_loss = running_loss / len(val_loader.dataset)
    val_acc = correct / total
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # Convert lists to numpy arrays for log_loss calculation
    all_probs = np.array(all_probs)
    all_labels = np.array(all_labels)
    log_loss_value = log_loss(all_labels, all_probs)

    return val_loss, val_acc, precision, recall, f1, log_loss_value


In [23]:
# Set the number of training epochs and early stopping patience
num_epochs = 300  # Total epochs for training
patience = 5      # Number of epochs with no improvement after which training will stop
counter = 0       # Counter to track epochs without improvement
best_val_loss = float('inf')  # Initialize the best validation loss to infinity
best_val_acc = 0.0  # Initialize the best validation accuracy to 0
best_metrics = {}   # Dictionary to store the best validation metrics

# Training loop
for epoch in range(num_epochs):
    # Train for one epoch and get training loss and accuracy
    train_loss, train_acc = train_epoch(model, train_loader)

    # Validate the model and get validation metrics
    val_loss, val_acc, precision, recall, f1, log_loss_value = validate_epoch(model, val_loader)

    # Check if the current validation loss is the best so far
    if val_loss < best_val_loss:
        best_val_loss = val_loss  # Update the best validation loss
        counter = 0  # Reset the counter as we have improvement

        # Save the model with the best validation loss
        torch.save(model.state_dict(), 'best_model.pth')

        # Update best validation accuracy and metrics
        best_val_acc = val_acc
        best_metrics = {
            "Accuracy": val_acc,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "Log Loss": log_loss_value
        }
    else:
        # Increment the counter if there is no improvement in validation loss
        counter += 1

    # Check if early stopping condition is met
    if counter >= patience:
        print("Early stopping triggered")
        break  # Exit the loop if early stopping is triggered

    # Print training and validation metrics for the current epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, Log Loss: {log_loss_value:.4f}")


Epoch [1/300]
Train Loss: 0.8107, Train Acc: 0.7297
Val Loss: 0.9853, Val Acc: 0.7413
Precision: 0.8209, Recall: 0.7413, F1 Score: 0.7268, Log Loss: 0.9853


KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("best_model.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>